# Import necessary libraries

In [1]:
import pandas as pd
import pycountry
import os

In [2]:
print(os.getcwd())

/Users/timoklein/Library/CloudStorage/OneDrive-Personal/Universiteit/Studie/M_Data_Science_and_Society/year_2/Thesis/Code/timoklein


# Preprocessing the datasets

## Preprocess the Technology Readiness Index dataset

In [7]:
## Load the Technology Readiness Index dataset into a DataFrame
df_tri = pd.read_csv('Data/Technology_Readiness_Index.csv')

# Only keep the columns 'Economy_Label', 'Category_Label', '2015_Index_Value' and '2016_Index_Value'
df_tri = df_tri[['Economy_Label', 'Category_Label', '2015_Index_Value', '2016_Index_Value']]

# Add a column with the average index value for 2015 and 2016
df_tri['Average_Index_Value'] = df_tri[['2015_Index_Value', '2016_Index_Value']].mean(axis=1)

# Delete the 2015 and 2016 index value columns
df_tri.drop(columns=['2015_Index_Value', '2016_Index_Value'], inplace=True)

# Pivot the dataframe
df_tri_cleaned = df_tri.pivot(index='Economy_Label', columns='Category_Label', values=['Average_Index_Value'])

# Flatten the multi-level column index
df_tri_cleaned.columns = ['_'.join(col).strip() for col in df_tri_cleaned.columns.values]

# Delete the row with 'Economy_Label'
df_tri_cleaned.reset_index(inplace=True)

# Rename the columns
df_tri_cleaned.columns = ['Country', 'Finance_access', 'ICT', 'Industry_activity', 'Overall_index', 'Research_and_development', 'Skills']

# Drop missing values
df_tri_cleaned.dropna(inplace=True)

# Change the 'Country' column to ISO codes
# Extract the 'Country' column as a list
country_names = df_tri_cleaned['Country'].tolist()

# Create a mapping of country names to ISO codes
country_to_iso = {}
for country in country_names:
    try:
        # Get the country object using the name
        country_obj = pycountry.countries.lookup(country)
        # Map country name to ISO code
        country_to_iso[country] = country_obj.alpha_3  # Using 3-letter ISO code
    except LookupError:
        # If the country is not found, handle it as needed
        print(f"Country not found: {country}")

# Manual mapping for countries that did not get an ISO code
manual_iso_mapping = {
    "Bolivia (Plurinational State of)": "BOL",
    "China, Hong Kong SAR": "HKG",
    "Congo, Dem. Rep. of the": "COD",
    "Cote d'Ivoire": "CIV",
    "Iran (Islamic Republic of)": "IRN",
    "Lao People's Dem. Rep.": "LAO",
    "Netherlands (Kingdom of the)": "NLD",
    "Switzerland, Liechtenstein": "CHE",  
    "Turkiye": "TUR",  
    "Venezuela (Bolivarian Rep. of)": "VEN"
}

# Update the mapping with the manual mappings
country_to_iso.update(manual_iso_mapping)

# Create a Series from the mapping to use for replacing values
iso_codes_series = pd.Series(country_to_iso)

# Replace the 'Country' column values with ISO codes
df_tri_cleaned['Country'] = df_tri_cleaned['Country'].replace(iso_codes_series)

df_tri_cleaned.head()

Country not found: Bolivia (Plurinational State of)
Country not found: China, Hong Kong SAR
Country not found: Congo, Dem. Rep. of the
Country not found: Cote d'Ivoire
Country not found: Iran (Islamic Republic of)
Country not found: Lao People's Dem. Rep.
Country not found: Netherlands (Kingdom of the)
Country not found: Switzerland, Liechtenstein
Country not found: Turkiye
Country not found: Venezuela (Bolivarian Rep. of)


,Country,Finance_access,ICT,Industry_activity,Overall_index,Research_and_development,Skills
0,AFG,0.15,0.00,0.25,0.00,0.00,0.10
1,ALB,0.60,0.50,0.50,0.40,0.10,0.45
2,DZA,0.50,0.25,0.20,0.30,0.30,0.40
3,AGO,0.50,0.10,0.25,0.15,0.05,0.20
4,ARG,0.40,0.55,0.60,0.55,0.30,0.65


In [17]:
df_tri_cleaned.describe()

,Finance_access,ICT,Industry_activity,Overall_index,Research_and_development,Skills
count,166.000000,166.000000,166.000000,166.000000,166.000000,166.000000
mean,0.628614,0.423494,0.574699,0.450904,0.246084,0.432229
std,0.180766,0.249768,0.193199,0.262648,0.240580,0.238875
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.500000,0.200000,0.400000,0.200000,0.050000,0.212500
50%,0.650000,0.450000,0.600000,0.400000,0.150000,0.400000
75%,0.750000,0.600000,0.700000,0.650000,0.400000,0.600000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Preprocess the Road Traffic Deaths Sex dataset

In [10]:
## Load the Road Traffic Deaths Sex dataset into a DataFrame
df_rtds = pd.read_csv('Data/Road_Traffic_Deaths_Sex.csv')

# Only keep the columns 'Unnamed: 0', 'Unnamed: 1', 'Estimated road traffic death rate (per 100 000 population)', 'Estimated road traffic death rate (per 100 000 population).1' and 'Estimated road traffic death rate (per 100 000 population).2'
df_rtds = df_rtds[['Unnamed: 0', 'Unnamed: 1', 'Estimated road traffic death rate (per 100 000 population)', 'Estimated road traffic death rate (per 100 000 population).1', 'Estimated road traffic death rate (per 100 000 population).2']]

# Rename the columns and delete the first row
df_rtds.columns = ['Country', 'Year', 'Total', 'Males', 'Females']
df_rtds = df_rtds.iloc[1:]

# Only keep the rows where the 'Year' column is '2015' or '2016'
df_rtds['Year'] = df_rtds['Year'].str.strip()
df_rtds = df_rtds[df_rtds['Year'].isin(['2015', '2016'])]

# Remove the numbers between square brackets
columns_to_clean = ['Total','Males','Females']

for col in columns_to_clean:
    df_rtds[col] = df_rtds[col].str.replace(r'\s*\[.*?\]', '', regex=True)

# Convert the death rate columns to numeric
df_rtds['Total'] = pd.to_numeric(df_rtds['Total'], errors='coerce')
df_rtds['Males'] = pd.to_numeric(df_rtds['Males'], errors='coerce')
df_rtds['Females'] = pd.to_numeric(df_rtds['Females'], errors='coerce')
df_rtds['Year'] = pd.to_numeric(df_rtds['Year'], errors='coerce')

# Group by 'Country' and calculate the mean for the last three columns for '2015' and '2016'
df_rtds = df_rtds.groupby('Country').mean()[['Total','Males','Females']]

# Normalize the last three columns 
# Convert columns to numeric type
columns_to_normalize = ['Total','Males','Females']

for col in columns_to_normalize:
    df_rtds[col] = pd.to_numeric(df_rtds[col])

# Min-Max Normalization directly in the existing DataFrame
for col in columns_to_normalize:
    df_rtds[col] = (df_rtds[col] - df_rtds[col].min()) / (df_rtds[col].max() - df_rtds[col].min())

# Reset the index so that "Country" becomes a regular column
df_rtds = df_rtds.reset_index()

# Change the 'Country' column to ISO codes
# Extract the 'Country' column as a list
country_names = df_rtds['Country'].tolist()

# Create a mapping of country names to ISO codes
country_to_iso = {}
for country in country_names:
    try:
        # Get the country object using the name
        country_obj = pycountry.countries.lookup(country)
        # Map country name to ISO code
        country_to_iso[country] = country_obj.alpha_3  # Using 3-letter ISO code
    except LookupError:
        # If the country is not found, handle it as needed
        print(f"Country not found: {country}")

# Manual mapping for countries that did not get an ISO code
manual_iso_mapping = {
    "Bolivia (Plurinational State of)": "BOL",
    "Cote d'Ivoire": "CIV",  
    "Democratic Republic of the Congo": "COD",
    "Iran (Islamic Republic of)": "IRN",
    "Micronesia (Federated States of)": "FSM",
    "Netherlands (Kingdom of the)": "NLD",
    "Republic of Korea": "KOR",
    "Turkiye": "TUR",  
    "Venezuela (Bolivarian Republic of)": "VEN"
}

# Update the mapping with the manual mappings
country_to_iso.update(manual_iso_mapping)

# Create a Series from the mapping to use for replacing values
iso_codes_series = pd.Series(country_to_iso)

# Replace the 'Country' column values with ISO codes
df_rtds['Country'] = df_rtds['Country'].replace(iso_codes_series)

df_rtds.head(10)


Country not found: Bolivia (Plurinational State of)
Country not found: Cote d'Ivoire
Country not found: Democratic Republic of the Congo
Country not found: Iran (Islamic Republic of)
Country not found: Micronesia (Federated States of)
Country not found: Netherlands (Kingdom of the)
Country not found: Republic of Korea
Country not found: Turkiye
Country not found: Venezuela (Bolivarian Republic of)


,Country,Total,Males,Females
0,AFG,0.371501,0.355590,0.240586
1,ALB,0.351145,0.324534,0.269874
2,DZA,0.538168,0.440994,0.573222
3,AGO,0.614504,0.478261,0.742678
4,ATG,0.000000,0.000000,0.000000
5,ARG,0.356234,0.347050,0.251046
6,ARM,0.412214,0.389752,0.349372
7,AUS,0.137405,0.121118,0.123431
8,AUT,0.138677,0.123447,0.125523
9,AZE,0.241730,0.236801,0.158996


In [20]:
df_rtds.describe()

,Total,Males,Females
count,183.000000,183.000000,183.000000
mean,0.434197,0.393956,0.366543
std,0.238905,0.215040,0.242767
min,0.000000,0.000000,0.000000
25%,0.241730,0.224379,0.174686
50%,0.408397,0.376553,0.303347
75%,0.630407,0.551242,0.544979
max,1.000000,1.000000,1.000000


## Preprocess the Road Traffic Deaths User dataset

In [14]:
## Read the Road Traffic Deaths Sex dataset into a DataFrame
df_rtdsfu = pd.read_csv('Data/Road_Traffic_Deaths_Sex.csv')

## Read the Road Traffic Deaths User dataset into a DataFrame
df_rtdu = pd.read_csv('Data/Road_Traffic_Deaths_User.csv')

# Only keep the columns 'Unnamed: 0' and 'Estimated number of road traffic deaths'
df_rtdsfu = df_rtdsfu[['Unnamed: 0','Unnamed: 1','Estimated number of road traffic deaths']]

# Rename the columns and delete the first row
df_rtdsfu.columns = ['Country', 'Year','Total']
df_rtdsfu = df_rtdsfu.iloc[1:]

# Only keep the rows where the 'Year' column is '2016'
df_rtdsfu['Year'] = df_rtdsfu['Year'].str.strip()
df_rtdsfu = df_rtdsfu[df_rtdsfu['Year'].isin(['2016'])]

# Remove the numbers between square brackets
df_rtdsfu['Total'] = df_rtdsfu['Total'].str.replace(r'\s*\[.*?\]', '', regex=True)

# Delete the 'Year' column
df_rtdsfu.drop(columns=['Year'], inplace=True)

# Only keep the columns 'Unnamed: 0','Unnamed: 1', 'Distribution of road traffic deaths by type of road user (%)' and 'Distribution of road traffic deaths by type of road user (%).3'
df_rtdu = df_rtdu[['Unnamed: 0','Unnamed: 1','Distribution of road traffic deaths by type of road user (%)', 'Distribution of road traffic deaths by type of road user (%).3']]

# Rename the columns and delete the first row
df_rtdu.columns = ['Country', 'Year','Passengers', 'Pedestrians']
df_rtdu = df_rtdu.iloc[1:]

# Only keep the rows where the 'Year' column is '2016'
df_rtdu['Year'] = df_rtdu['Year'].str.strip()
df_rtdu = df_rtdu[df_rtdu['Year'].isin(['2016'])]

# Remove the 'h' from the 'Passengers' column
df_rtdu['Passengers'] = df_rtdu['Passengers'].str.replace('h', '', regex=False)

# Delete missing values
df_rtdu.dropna(inplace=True)

# Add the column 'Estimated_number_of_road_traffic_deaths' from the 'df_rtdsfu' DataFrame to the 'df_rtdu' DataFrame
df_rtdu_cleaned = pd.merge(df_rtdu, df_rtdsfu, on='Country', how='inner')

# Convert the 'Passengers', 'Pedestrians' and 'Total' columns to numeric
df_rtdu_cleaned['Passengers'] = pd.to_numeric(df_rtdu_cleaned['Passengers'], errors='coerce')
df_rtdu_cleaned['Pedestrians'] = pd.to_numeric(df_rtdu_cleaned['Pedestrians'], errors='coerce')
df_rtdu_cleaned['Total'] = pd.to_numeric(df_rtdu_cleaned['Total'], errors='coerce')

# Add a column with the estimated number of road traffic deaths for passengers and pedestrians
df_rtdu_cleaned['Passengers'] = df_rtdu_cleaned['Passengers']/100 * df_rtdu_cleaned['Total']
df_rtdu_cleaned['Pedestrians'] = df_rtdu_cleaned['Pedestrians']/100 * df_rtdu_cleaned['Total']

# Delete the 'Year' and 'Total' column
df_rtdu_cleaned.drop(columns=['Year','Total'], inplace=True)

# Convert columns to numeric type
columns_to_normalize = ['Passengers','Pedestrians']

for col in columns_to_normalize:
    df_rtdu_cleaned[col] = pd.to_numeric(df_rtdu_cleaned[col])

# Min-Max Normalization directly in the existing DataFrame
for col in columns_to_normalize:
    df_rtdu_cleaned[col] = (df_rtdu_cleaned[col] - df_rtdu_cleaned[col].min()) / (df_rtdu_cleaned[col].max() - df_rtdu_cleaned[col].min())

# Change the 'Country' column to ISO codes
# Extract the 'Country' column as a list
country_names = df_rtdu_cleaned['Country'].tolist()

# Create a mapping of country names to ISO codes
country_to_iso = {}
for country in country_names:
    try:
        # Get the country object using the name
        country_obj = pycountry.countries.lookup(country)
        # Map country name to ISO code
        country_to_iso[country] = country_obj.alpha_3  # Using 3-letter ISO code
    except LookupError:
        # If the country is not found, handle it as needed
        print(f"Country not found: {country}")

# Manual mapping for countries that did not get an ISO code
manual_iso_mapping = {
    "Bolivia (Plurinational State of)": "BOL",
    "Cote d'Ivoire": "CIV",  # Côte d'Ivoire
    "Democratic Republic of the Congo": "COD",
    "Iran (Islamic Republic of)": "IRN",
    "Micronesia (Federated States of)": "FSM",
    "Netherlands (Kingdom of the)": "NLD",
    "Republic of Korea": "KOR"  # Use KOR for South Korea
}

# Update the mapping with the manual mappings
country_to_iso.update(manual_iso_mapping)

# Create a Series from the mapping to use for replacing values
iso_codes_series = pd.Series(country_to_iso)

# Replace the 'Country' column values with ISO codes
df_rtdu_cleaned['Country'] = df_rtdu_cleaned['Country'].replace(iso_codes_series)

df_rtdu_cleaned.tail(10)

Country not found: Bolivia (Plurinational State of)
Country not found: Cote d'Ivoire
Country not found: Democratic Republic of the Congo
Country not found: Iran (Islamic Republic of)
Country not found: Micronesia (Federated States of)
Country not found: Netherlands (Kingdom of the)
Country not found: Republic of Korea


,Country,Passengers,Pedestrians
107,THA,0.287871,0.080806
108,TON,0.000286,0.000237
109,TTO,0.002239,0.002431
110,TUN,0.015497,0.030237
111,UGA,0.112620,0.225192
112,UKR,0.091326,0.120846
113,ARE,0.021335,0.019314
114,GBR,0.021392,0.022665
115,TZA,0.285799,0.230171
116,USA,0.351087,0.301153


In [87]:
df_rtdu_cleaned.describe()

,Passengers,Pedestrians
count,117.000000,117.000000
mean,0.044421,0.059473
std,0.110587,0.130570
min,0.000000,0.000000
25%,0.003017,0.002754
50%,0.009310,0.015333
75%,0.042836,0.046221
max,1.000000,1.000000


## Preprocess the Moral Machine dataset

In [272]:
# Load the 'ExtendedSessionID' column from the Moral Machine dataset into a DataFrame
extendedsessionid = pd.read_csv('Data/SharedResponses.csv', usecols=['ExtendedSessionID'])

In [273]:
# Explore the data structure of the 'ExtendedSessionID' column
print(extendedsessionid.head())
print(extendedsessionid.info())
print(extendedsessionid.describe())
print(extendedsessionid.shape)

               ExtendedSessionID
0    32757157_6999801415950060.0
1        1043988516_3525281295.0
2  -1613944085_422160228641876.0
3   1425316635_327833569077076.0
4  -1683127088_785070916172117.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70332355 entries, 0 to 70332354
Data columns (total 1 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   ExtendedSessionID  object
dtypes: object(1)
memory usage: 536.6+ MB
None
                   ExtendedSessionID
count                       70332355
unique                       3564584
top     -92417687_1460359671541513.0
freq                           29542
(70332355, 1)


In [274]:
# Filter out the sessions that have more than 26 responses
session_counts = extendedsessionid['ExtendedSessionID'].value_counts()
ids_to_keep = session_counts[session_counts <= 26].index

In [ ]:
df_mm = pd.read_csv('Data/SharedResponses.csv', chunksize=100_000, dtype=str, low_memory=False)

cleaned_chunks = []
for i, chunk in enumerate(df_mm):
    print(f"Processing chunk {i+1}")
    
    # Cleaning 
    chunk_cleaned = chunk[chunk['ExtendedSessionID'].isin(ids_to_keep)]
    chunk_cleaned = chunk_cleaned.drop(columns=['ScenarioOrder', 'Intervention', 'ScenarioType', 'DefaultChoice', 'NonDefaultChoice', 'DefaultChoiceIsOmission', 'Template','DescriptionShown','LeftHand'])
    chunk_cleaned.rename(columns={'UserCountry3': 'Country'}, inplace=True)
    chunk_cleaned = chunk_cleaned.dropna()
    chunk_cleaned = chunk_cleaned.drop_duplicates()
    chunk_cleaned = chunk_cleaned.merge(df_tri_cleaned, on='Country', how='inner').merge(df_rtds, on='Country', how='inner').merge(df_rtdu_cleaned, on='Country', how='inner')

    chunk_cleaned[['ResponseID', 'ExtendedSessionID', 'UserID', 'AttributeLevel','ScenarioTypeStrict']] = chunk_cleaned[['ResponseID', 'ExtendedSessionID', 'UserID', 'AttributeLevel','ScenarioTypeStrict']].astype(str)
    chunk_cleaned[[
    "PedPed", "Barrier", "CrossingSignal", "NumberOfCharacters", "DiffNumberOfCharacters",
    "Man", "Woman", "Pregnant", "Stroller", "OldMan", "OldWoman", "Boy", "Girl", "Homeless",
    "LargeWoman", "LargeMan", "Criminal", "MaleExecutive", "FemaleExecutive", "FemaleAthlete",
    "MaleAthlete", "FemaleDoctor", "MaleDoctor", "Dog and Cat", "Saved"]] = chunk_cleaned[["PedPed", "Barrier", "CrossingSignal", "NumberOfCharacters", "DiffNumberOfCharacters",
    "Man", "Woman", "Pregnant", "Stroller", "OldMan", "OldWoman", "Boy", "Girl", "Homeless",
    "LargeWoman", "LargeMan", "Criminal", "MaleExecutive", "FemaleExecutive", "FemaleAthlete",
    "MaleAthlete", "FemaleDoctor", "MaleDoctor", "Dog and Cat", "Saved"]].astype(int)
    chunk_cleaned[["Finance_access", "ICT", "Industry_activity", "Overall_index", "Research_and_development", "Skills",
    "Total", "Males", "Females", "Passengers", "Pedestrians"]] = chunk_cleaned[["Finance_access",
    "ICT", "Industry_activity", "Overall_index", "Research_and_development", "Skills",
    "Total", "Males", "Females", "Passengers", "Pedestrians"]].astype(float)

    # Append cleaned chunk
    cleaned_chunks.append(chunk_cleaned)
    
    print(f"Finished processing chunk {i+1}")

print("All chunks have been processed")

In [ ]:
cleaned_data = pd.concat(cleaned_chunks, ignore_index=True)

In [ ]:
# Merge cleaned_data with the other dataframes
merged_df = cleaned_data.merge(df_tri_cleaned, on='Country', how='inner').merge(df_rtds, on='Country', how='inner').merge(df_rtdu_cleaned, on='Country', how='inner')